In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import numpy as np
from sklearn import model_selection as ms
from pandas.api.types import CategoricalDtype
pd.set_option("display.max_rows", 15)
get_ipython().run_line_magic('matplotlib', 'inline')

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
no_of_lines = 409349 #train
train_size = int(no_of_lines * 0.1)
no_of_lines = 423984 #test
test_size = int(no_of_lines * 0.1)
print(train_size, test_size)

40934 42398


In [4]:
class dataset:
  col_names = ['sIP', 'dIp', 'sPort', 'pro', 'packets', 'bytes', 'flags', 'sTime', 'durat', 'eTime', 'sen', 'in', 'out', 'nhIP',
               'cla', 'type', 'iTy','iCo', 'initialF', 'sessionF', 'attribut', 'appli', 'status', 'taxonomy', 'label', 'heuristic', 'distance', 'nbDetectors']

  sample_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/20170827_mawilab_flow_001.csv", 
                            header = 0, names = col_names, nrows = train_size)
  sample_data.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/sample_train.pkl")

In [5]:
class dataset:
  col_names = ['sIP', 'dIp', 'sPort', 'pro', 'packets', 'bytes', 'flags', 'sTime', 'durat', 'eTime', 'sen', 'in', 'out', 'nhIP',
               'cla', 'type', 'iTy','iCo', 'initialF', 'sessionF', 'attribut', 'appli', 'status', 'taxonomy', 'label', 'heuristic', 'distance', 'nbDetectors']

  sample_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/20170827_mawilab_flow_003.csv", 
                            header = 0, names = col_names, nrows = test_size)
  sample_data.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/sample_test.pkl")

In [6]:
def process_train(df):
  df = df.loc[:,('pro', 'packets', 'bytes', 'durat', 'status')].copy()
  return df
dataset.mawi_train = process_train(pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/sample_train.pkl'))
dataset.mawi_train_label = dataset.mawi_train.status.copy()

dataset.mawi_train

,pro,packets,bytes,durat,status
103.219.67.21,6,1,40,0,normal
163.158.38.208,6,1,68,0,normal
107.90.51.215,6,1,290,0,normal
163.158.39.10,6,1,52,0,normal
150.35.0.77,6,1,40,0,anomaly
...,...,...,...,...,...
108.229.55.54,1,1,32,0,normal
203.205.70.80,6,1,1480,0,anomaly
23.209.241.38,6,1,1500,0,normal
203.104.241.234,6,1,1500,0,normal


In [7]:
def process_test(df):
  df = df.loc[:,('pro', 'packets', 'bytes', 'durat', 'status')].copy()
  return df
dataset.mawi_test = process_test(pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/sample_test.pkl'))
dataset.mawi_test_label = dataset.mawi_test.status.copy()

dataset.mawi_test

,pro,packets,bytes,durat,status
163.158.38.28,6,1,40,0.0,normal
163.158.38.28,6,1,40,0.0,normal
203.205.66.188,6,1,40,0.0,anomaly
38.6.59.58,6,1,1500,0.0,anomaly
92.193.158.85,1,1,32,0.0,normal
...,...,...,...,...,...
178.39.114.118,6,1,44,0.0,normal
203.205.70.93,6,1,40,0.0,normal
172.221.157.107,17,1,36,0.0,anomaly
203.205.73.47,47,1,83,0.0,anomaly


In [8]:
class one_hot:
  one_hot_encoding = pd.get_dummies(dataset.mawi_test)

In [9]:
one_hot.one_hot_encoding

,pro,packets,bytes,durat,status_anomaly,status_normal
163.158.38.28,6,1,40,0.0,0,1
163.158.38.28,6,1,40,0.0,0,1
203.205.66.188,6,1,40,0.0,1,0
38.6.59.58,6,1,1500,0.0,1,0
92.193.158.85,1,1,32,0.0,0,1
...,...,...,...,...,...,...
178.39.114.118,6,1,44,0.0,0,1
203.205.70.93,6,1,40,0.0,0,1
172.221.157.107,17,1,36,0.0,1,0
203.205.73.47,47,1,83,0.0,1,0


In [ ]:
category_variables = ['status']

for cv in category_variables:
  dataset.mawi_train[cv] = dataset.mawi_train[cv]. astype("category")
  dataset.mawi_test[cv] = dataset.mawi_test[cv].astype(CategoricalDtype(categories = dataset.mawi_train[cv].cat.categories))

In [ ]:
dummy_variables_2labels = category_variables

class preprocessing:
  mawi_train_2labels = pd.get_dummies(dataset.mawi_train, columns = dummy_variables_2labels, prefix = dummy_variables_2labels)
  mawi_test_2labels = pd.get_dummies(dataset.mawi_test, columns = dummy_variables_2labels, prefix = dummy_variables_2labels)

  mawi_train_2labels_y = dataset.mawi_train["status"].copy()
  mawi_test_2labels_y = dataset.mawi_test["status"].copy()

In [ ]:
preprocessing.mawi_train_2labels.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl")
preprocessing.mawi_train_2labels_y.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels_y(001).pkl")
preprocessing.mawi_test_2labels.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(003).pkl")
preprocessing.mawi_test_2labels_y.to_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels_y(003).pkl")